In [3]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

from sparknlp.base import DocumentAssembler
from sparknlp.annotator import T5Transformer

import warnings
warnings.filterwarnings('ignore')

import os 
os.environ['PYSPARK_PYTHON'] = "/root/anaconda3/bin/python"     # 设置了运行 PySpark 的 Python 解释器的环境变量



In [4]:
from kafka import KafkaConsumer
import json 

broker = 'localhost:64050'
consumer = KafkaConsumer('q1_3', 
        bootstrap_servers=broker, 
        value_deserializer=lambda m: json.loads(m.decode('utf-8')), 
        consumer_timeout_ms=100000,
        # auto_offset_reset='earliest', enable_auto_commit=False
    )

In [9]:
# spark2 = SparkSession.builder\
#     .config(\
#         "spark.jars",  \
#         "/shareddata/lab09/kafka-clients-3.5.0.jar, \
#         /shareddata/lab09/spark-sql-kafka-0-10_2.12-3.5.0.jar, \
#         /shareddata/lab09/spark-token-provider-kafka-0-10_2.12-3.5.0.jar, \
#         /shareddata/lab09/commons-pool2-2.12.0.jar") \
#     .config("spark.executorEnv.PYSPARK_PYTHON","/root/anaconda3/bin/python")\
#     .config("spark.executor.memory", "16g") \
#     .config("spark.driver.memory", "16g") \
#     .config("spark.log.level", "WARN") \
#     .appName("q1_3").getOrCreate()

# df = spark2.readStream.format("kafka")\
# 	.option("kafka.bootstrap.servers", "localhost:64050")\
# 	.option("subscribe", "q1_3")\
# 	.option("startingOffsets", "earliest")\
#     .load()
# df.printSchema()

# df1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","CAST (timestamp AS TIMESTAMP)")
# schema = "input String, output String"
# result = df1.selectExpr("key", "from_json(value, '{}') AS data".format(schema), "timestamp")
# q3_query = result.writeStream.queryName("QA").format("memory").outputMode("update").trigger(processingTime="5 seconds").start()

# data_df = spark2.sql("SELECT data as text from QA")
# data_df.show(5)

In [5]:
spark1 = sparknlp.start()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a11de920-206f-44bc-88e5-b7bbdbe910ed;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
:: resolution report :: resolve 113ms :: artifacts dl 6ms
	:: modules in use:
	com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-a11de920-206f-44bc-88e5-b7bbdbe9

In [ ]:

MODEL_NAME = "flan-t5-small"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"


In [9]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("summary")


pipeline = Pipeline().setStages([document_assembler, T5])



NameError: name 'MODEL_NAME' is not defined

In [8]:
for message in consumer:
    input = message.value["input"]
    output = message.value["output"]
    # print(input)
    data_df = spark1.createDataFrame([input]).toDF("text")
    results = pipeline.fit(data_df).transform(data_df)
    results.select("question.result").show(truncate=False)


{'input': 'question: What is a main advantage of the Rankine cycle? context: One of the principal advantages the Rankine cycle holds over others is that during the compression stage relatively little work is required to drive the pump, the working fluid being in its liquid phase at this point. By condensing the fluid, the work required by the pump consumes only 1% to 3% of the turbine power and contributes to a much higher efficiency for a real cycle. The benefit of this is lost somewhat due to the lower heat addition temperature. Gas turbines, for instance, have turbine entry temperatures approaching 1500 °C. Nonetheless, the efficiencies of actual large steam cycles and large modern gas turbines are fairly well matched.[citation needed]', 'output': 'relatively little work is required to drive the pump,'}
